In [6]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from keys import api_key
from pytz import timezone

def get_flight_data(icao_list):


    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    flight_items = []

    for icao in icao_list:
        # the api can only make 12 hour calls, therefore, 2 12 hour calls make a full day
        # using the nested lists below we can make a morning call and extract the data
        # then make an afternoon call and extract the data
        times = [["00:00","11:59"],
                 ["12:00","23:59"]]

        for time in times:

            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"withLeg":"true",
                        "direction":"Arrival",
                        "withCancelled":"false",
                        "withCodeshared":"true",
                        "withCargo":"false",
                        "withPrivate":"false"}

            headers = {
              'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
              'x-rapidapi-key': api_key
              }

            response = requests.get(url, headers=headers, params=querystring)

            flights_json = response.json()

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flights_json["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "departure_airport_name": item["departure"]["airport"].get("name", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieved_at": retrieval_time
                }

                flight_items.append(flight_item)

    flights_df = pd.DataFrame(flight_items)
    flights_df["scheduled_arrival_time"] = flights_df["scheduled_arrival_time"].str[:-6]
    flights_df["scheduled_arrival_time"] = pd.to_datetime(flights_df["scheduled_arrival_time"])
    flights_df["data_retrieved_at"] = pd.to_datetime(flights_df["data_retrieved_at"])

    return flights_df

In [14]:
icao_list = ["EDDB", "EDDH", "EDDM"]

flights_to_db = get_flight_data(icao_list)
flights_to_db

,arrival_airport_icao,departure_airport_icao,departure_airport_name,scheduled_arrival_time,flight_number,data_retrieved_at
0,EDDB,OTHH,Doha,2025-02-25 06:55:00,QR 79,2025-02-24 16:07:21
1,EDDB,LFSB,Bâle/Mulhouse,2025-02-25 07:40:00,U2 1185,2025-02-24 16:07:21
2,EDDB,LFSB,Bâle/Mulhouse,2025-02-25 07:40:00,DS 1185,2025-02-24 16:07:21
3,EDDB,EDDK,Cologne,2025-02-25 07:40:00,EW 2,2025-02-24 16:07:21
4,EDDB,EDDS,Stuttgart,2025-02-25 07:45:00,EW 2000,2025-02-24 16:07:21
...,...,...,...,...,...,...
737,EDDM,LEMD,Madrid,2025-02-25 22:40:00,IB 747,2025-02-24 16:07:22
738,EDDM,EGLL,London,2025-02-25 22:45:00,BA 956,2025-02-24 16:07:22
739,EDDM,EGLL,London,2025-02-25 22:45:00,BA 958,2025-02-24 16:07:22
740,EDDM,EGLL,London,2025-02-25 23:05:00,LH 2483,2025-02-24 16:07:22


# Push the "flights_to_db" to the empty "flights" table in the database

In [35]:
from keys import MySQL_pass

schema = "gans_data"
host = "127.0.0.1"
user = "root"
password = MySQL_pass
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [39]:
flights_to_db.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)

742

### SUCCESS!